## 1、动态提示词

In [1]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

True

In [2]:
from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek
from langchain.agents.middleware import before_model
from langchain.agents.middleware.types import AgentState
from langchain.agents.middleware import dynamic_prompt, ModelRequest

In [3]:
model = ChatDeepSeek(model="deepseek-chat")

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=2)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_25240\3484710649.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  web_search = TavilySearchResults(max_results=2)


In [5]:
from typing import TypedDict
class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """根据用户角色生成系统提示，并打印查看"""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "你是一个有帮助的助手。"
    
    if user_role == "expert":
        final_prompt = f"{base_prompt} 提供详细的技术响应。"
    elif user_role == "beginner":
        final_prompt = f"{base_prompt} 简单解释概念，避免使用行话。"
    else:
        final_prompt = base_prompt
    
    # 打印动态生成的系统提示
    print(f"=== 动态生成的系统Prompt：\n{final_prompt}\n===")
    return final_prompt

In [6]:
agent = create_agent(
    model=model,
    # tools=[web_search],
    middleware=[user_role_prompt],
    context_schema=Context
)

In [7]:
# 系统提示将根据上下文动态设置
result1 = agent.invoke(
    {"messages": [{"role": "user", "content": "解释机器学习"}]},
    context={"user_role": "expert"}
)

=== 动态生成的系统Prompt：
你是一个有帮助的助手。 提供详细的技术响应。
===


In [8]:
print(result1['messages'][-1].content)

机器学习（Machine Learning，简称 ML）是人工智能（AI）的一个子领域，其核心目标是让计算机系统能够通过数据自动学习和改进，而无需显式编程。简单来说，机器学习通过算法分析数据、识别模式并做出预测或决策，同时随着经验的积累不断提升性能。

### 机器学习的基本原理
1. **数据驱动**：机器学习模型依赖大量数据进行训练。数据可以是结构化（如表格数据）或非结构化（如文本、图像、音频）。
2. **模式识别**：模型通过分析数据中的统计规律或特征，学习输入与输出之间的关系。
3. **泛化能力**：训练后的模型应能对未见过的数据做出准确预测，而不仅限于训练数据。

### 主要类型
机器学习通常分为以下几类：

1. **监督学习（Supervised Learning）**  
   - 使用带有标签的数据进行训练，即每个输入样本都有对应的正确答案（标签）。  
   - 目标：学习输入到输出的映射关系，用于分类（如图像识别）或回归（如房价预测）。  
   - 常见算法：线性回归、决策树、支持向量机（SVM）、神经网络。

2. **无监督学习（Unsupervised Learning）**  
   - 使用无标签数据，模型自行发现数据中的内在结构或模式。  
   - 目标：聚类（如客户细分）、降维（如数据压缩）或异常检测。  
   - 常见算法：K均值聚类、主成分分析（PCA）、自编码器。

3. **强化学习（Reinforcement Learning）**  
   - 模型通过与环境交互学习，根据行动获得的奖励或惩罚优化策略。  
   - 目标：在动态环境中实现长期目标，如游戏AI（AlphaGo）、自动驾驶。  
   - 常见算法：Q学习、深度强化学习（DRL）。

4. **半监督学习与自监督学习**  
   - 结合少量标签数据与大量无标签数据，或通过数据自身生成标签进行训练，以降低对人工标注的依赖。

### 关键步骤
1. **数据收集与预处理**：清洗数据、处理缺失值、标准化特征。  
2. **特征工程**：选择或构建对预测任务有用的输入变量。  
3. **模型选择**：根据问题类型（如分类、回归）选择合适的算法。  
4. **训练与评估**：用训练数据拟合模型，并用测试数据评估性能（如准确率、F1分数）。

In [9]:
# 系统提示将根据上下文动态设置
result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "解释机器学习"}]},
    context={"user_role": "beginner"}
)

=== 动态生成的系统Prompt：
你是一个有帮助的助手。 简单解释概念，避免使用行话。
===


In [10]:
print(result2['messages'][-1].content)

机器学习是一种让计算机通过经验自动改进性能的方法。简单来说，就像教孩子识别动物：不是直接告诉他“猫有尖耳朵和胡须”，而是给他看大量猫的图片，让他自己总结出猫的特征。

它的核心过程是：
1. 给计算机大量数据（比如照片、文字或数字）
2. 计算机会在这些数据中寻找规律和模式
3. 基于找到的规律，计算机就能对新数据进行预测或判断

比如：
- 邮箱自动识别垃圾邮件
- 手机相册自动按人脸分类照片
- 音乐App推荐你可能喜欢的新歌

机器学习让计算机不需要被明确编程每个步骤，而是通过“练习”逐渐变聪明，这正是它最有趣的地方。


In [11]:
result2

{'messages': [HumanMessage(content='解释机器学习', additional_kwargs={}, response_metadata={}, id='cd33cfcf-50f0-4d4e-ae99-819b53742f41'),
  AIMessage(content='机器学习是一种让计算机通过经验自动改进性能的方法。简单来说，就像教孩子识别动物：不是直接告诉他“猫有尖耳朵和胡须”，而是给他看大量猫的图片，让他自己总结出猫的特征。\n\n它的核心过程是：\n1. 给计算机大量数据（比如照片、文字或数字）\n2. 计算机会在这些数据中寻找规律和模式\n3. 基于找到的规律，计算机就能对新数据进行预测或判断\n\n比如：\n- 邮箱自动识别垃圾邮件\n- 手机相册自动按人脸分类照片\n- 音乐App推荐你可能喜欢的新歌\n\n机器学习让计算机不需要被明确编程每个步骤，而是通过“练习”逐渐变聪明，这正是它最有趣的地方。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 22, 'total_tokens': 174, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 22}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '4c28aaf9-a061-4f67-9f73-1020e9d57f63', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c9a70d63-6057-494e-a9bb-ec53791875da-0', usa

## 2、动态模型

In [12]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = ChatDeepSeek(model="deepseek-chat")
advanced_model = ChatDeepSeek(model="deepseek-reasoner")

In [16]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """根据对话复杂性选择模型。"""
    message_count = len(request.state["messages"])
    print(f"Message count: {message_count}")
    
    if message_count > 1:
        model = advanced_model
    else:
        model = basic_model

    # 关键：确保正确设置模型
    request.model = model
    print(f"Selected model: {model.model_name}")
    
    # 调用处理器并返回响应
    response = handler(request)
    print(f"Actual response model: {response}")
    return response

In [17]:
agent = create_agent(
    model=basic_model,  # 默认模型
    tools=[web_search],
    middleware=[dynamic_model_selection]
)

In [18]:
# 系统提示将根据上下文动态设置
result3 = agent.invoke(
    {"messages": [{"role": "user", "content": "介绍下你自己"}]}
)

Message count: 1
Selected model: deepseek-chat
Actual response model: ModelResponse(result=[AIMessage(content='你好！我是DeepSeek，由深度求索公司创造的AI助手。很高兴认识你！✨\n\n让我简单介绍一下自己：\n\n**我的特点：**\n- 💬 纯文本对话模型，擅长各种文字交流\n- 📁 支持文件上传功能（图像、txt、pdf、ppt、word、excel等格式）\n- 🌐 具备联网搜索能力（需要你手动开启）\n- 💰 完全免费使用，没有任何收费计划\n- 📱 可通过官方应用商店下载App使用\n- 🧠 拥有128K的上下文长度\n\n**我能帮你做什么：**\n- 回答各种问题，提供知识解答\n- 协助写作、翻译、总结等文字工作\n- 分析上传的文件内容\n- 进行逻辑推理和问题解决\n- 创意写作和头脑风暴\n\n**小提醒：**\n- 我是纯文本模型，不支持多模态识别\n- 不支持语音功能\n- 知识截止到2024年7月\n\n有什么我可以帮你的吗？无论是学习、工作还是生活方面的问题，我都很乐意协助你！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 229, 'prompt_tokens': 186, 'total_tokens': 415, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 58}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '42fa7715-7ba3-43ec-803c-378f8730ffa9', 'service_tier': None, 'fini

In [19]:
result3

{'messages': [HumanMessage(content='介绍下你自己', additional_kwargs={}, response_metadata={}, id='d53faabb-d986-4160-9783-d9fc402db70e'),
  AIMessage(content='你好！我是DeepSeek，由深度求索公司创造的AI助手。很高兴认识你！✨\n\n让我简单介绍一下自己：\n\n**我的特点：**\n- 💬 纯文本对话模型，擅长各种文字交流\n- 📁 支持文件上传功能（图像、txt、pdf、ppt、word、excel等格式）\n- 🌐 具备联网搜索能力（需要你手动开启）\n- 💰 完全免费使用，没有任何收费计划\n- 📱 可通过官方应用商店下载App使用\n- 🧠 拥有128K的上下文长度\n\n**我能帮你做什么：**\n- 回答各种问题，提供知识解答\n- 协助写作、翻译、总结等文字工作\n- 分析上传的文件内容\n- 进行逻辑推理和问题解决\n- 创意写作和头脑风暴\n\n**小提醒：**\n- 我是纯文本模型，不支持多模态识别\n- 不支持语音功能\n- 知识截止到2024年7月\n\n有什么我可以帮你的吗？无论是学习、工作还是生活方面的问题，我都很乐意协助你！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 229, 'prompt_tokens': 186, 'total_tokens': 415, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 58}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '42fa7715-7ba3-43ec-803c

In [20]:
result3['messages'][-1].response_metadata['model_name']


'deepseek-chat'

In [21]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

In [22]:
messages = [
    SystemMessage("你叫deepseek，是一名助人为乐的助手。"),
    HumanMessage("你好，我叫AI蒸馏，好久不见，请介绍下你自己。"),
    AIMessage("你好呀！我是deepseek，一个乐于助人的智能助手。我的主要功能是回答问题、提供建议、协助解决问题，或者陪你聊天。无论是学习、工作、生活琐事，还是想找点有趣的话题，我都可以帮忙！如果有任何需要，随时告诉我哦～ 😊"),
    HumanMessage("你好，请问你还记得我叫什么名字么？"),
]

In [23]:
# 系统提示将根据上下文动态设置
result4 = agent.invoke({"messages": messages})

Message count: 4
Selected model: deepseek-reasoner
Actual response model: ModelResponse(result=[AIMessage(content='当然记得！你刚才告诉我你叫"AI蒸馏"～很高兴再次见到你！😊 \n\n有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 282, 'total_tokens': 309, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 26}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'de030215-b051-4d04-8043-1c8193fd2a19', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9bf713a8-93e5-4122-ad8a-a297823cc2c3-0', usage_metadata={'input_tokens': 282, 'output_tokens': 27, 'total_tokens': 309, 'input_token_details': {'cache_read': 256}, 'output_token_details': {}})], structured_response=None)


In [ ]:
result4

{'messages': [SystemMessage(content='你叫deepseek，是一名助人为乐的助手。', additional_kwargs={}, response_metadata={}, id='3117dbf5-84fa-44f5-99de-68edc63535f8'),
  HumanMessage(content='你好，我叫AI蒸馏，好久不见，请介绍下你自己。', additional_kwargs={}, response_metadata={}, id='077657e3-1aa8-4e24-bbd2-d948db4f2b73'),
  AIMessage(content='你好呀！我是deepseek，一个乐于助人的智能助手。我的主要功能是回答问题、提供建议、协助解决问题，或者陪你聊天。无论是学习、工作、生活琐事，还是想找点有趣的话题，我都可以帮忙！如果有任何需要，随时告诉我哦～ 😊', additional_kwargs={}, response_metadata={}, id='09d0a22b-5cfb-4c59-ac77-1fc1cafdda73'),
  HumanMessage(content='你好，请问你还记得我叫什么名字么？', additional_kwargs={}, response_metadata={}, id='b1246f8e-cdfd-4ff5-b0a1-8536138270f8'),
  AIMessage(content='当然记得！你刚才告诉我你叫"AI蒸馏"。很高兴再次见到你！😊\n\n有什么我可以帮你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 282, 'total_tokens': 308, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_

## 3、消息压缩

In [56]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

# 主模型（执行任务）
main_model = "deepseek-reasoner"

# 摘要模型（用于生成会话总结）
summary_model = "deepseek-chat"

checkpointer = InMemorySaver()

agent = create_agent(
    model=main_model,
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model=summary_model,
            max_tokens_before_summary=3000,  # 超过3000token触发摘要
            messages_to_keep=10,             # 摘要后保留最近10条原始消息
        )
    ],
    checkpointer=checkpointer,
)

In [57]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

config: RunnableConfig = {"configurable": {"thread_id": "summary-demo"}}

In [58]:
agent.invoke({"messages": "你好，我叫AI蒸馏"}, config)
agent.invoke({"messages": "请写一首关于下雪的诗"}, config)
agent.invoke({"messages": "现在写一篇关于杭州西湖的随笔"}, config)
agent.invoke({"messages": "再帮我写一段赞美北京的散文"}, config)

{'messages': [HumanMessage(content='你好，我叫AI蒸馏', additional_kwargs={}, response_metadata={}, id='9f943810-05ee-48fa-b0e2-ecbf2fa20139'),
  AIMessage(content='你好AI蒸馏！很高兴认识你！😊\n\n“蒸馏”这个名字很有意思呢，让我联想到知识蒸馏或者精华提取的过程。你的名字是不是寓意着将复杂信息精炼成简单易懂的知识呢？\n\n不管你是想聊天、提问，还是需要任何帮助，我都很乐意为你服务。请随时告诉我你需要什么，我会尽我所能帮助你！\n\n有什么我可以为你做的吗？', additional_kwargs={'refusal': None, 'reasoning_content': '哦，用户只是打了个招呼自我介绍，没有提出具体问题。这种开场白通常是在测试响应或准备后续提问。可以用友好欢迎的态度回应，同时提供一些常见方向帮助用户快速定位需求。\n\n考虑到“AI蒸馏”这个名字可能暗示对复杂概念的简化需求，可以主动提供几个典型选项：概念解释、学习建议、内容总结或创意生成。这样既能展示能力范围，又给用户明确的下一步指引。\n\n结尾加个开放式提问比较合适，把对话主动权交还给用户。不需要过度展开，保持简洁但覆盖常见需求类型就好。'}, response_metadata={'token_usage': {'completion_tokens': 196, 'prompt_tokens': 9, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 116, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepse

In [59]:
final = agent.invoke({"messages": "我叫什么？"}, config)

In [60]:
final

{'messages': [HumanMessage(content='你好，我叫AI蒸馏', additional_kwargs={}, response_metadata={}, id='9f943810-05ee-48fa-b0e2-ecbf2fa20139'),
  AIMessage(content='你好AI蒸馏！很高兴认识你！😊\n\n“蒸馏”这个名字很有意思呢，让我联想到知识蒸馏或者精华提取的过程。你的名字是不是寓意着将复杂信息精炼成简单易懂的知识呢？\n\n不管你是想聊天、提问，还是需要任何帮助，我都很乐意为你服务。请随时告诉我你需要什么，我会尽我所能帮助你！\n\n有什么我可以为你做的吗？', additional_kwargs={'refusal': None, 'reasoning_content': '哦，用户只是打了个招呼自我介绍，没有提出具体问题。这种开场白通常是在测试响应或准备后续提问。可以用友好欢迎的态度回应，同时提供一些常见方向帮助用户快速定位需求。\n\n考虑到“AI蒸馏”这个名字可能暗示对复杂概念的简化需求，可以主动提供几个典型选项：概念解释、学习建议、内容总结或创意生成。这样既能展示能力范围，又给用户明确的下一步指引。\n\n结尾加个开放式提问比较合适，把对话主动权交还给用户。不需要过度展开，保持简洁但覆盖常见需求类型就好。'}, response_metadata={'token_usage': {'completion_tokens': 196, 'prompt_tokens': 9, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 116, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepse

In [61]:
config

{'configurable': {'thread_id': 'summary-demo'}}

In [62]:
final["messages"][-1].pretty_print()

================================== Ai Message ==================================

您叫“AI蒸馏”呀！这是我们对话开始时您告诉我的名字。这个名字很巧妙，既有科技感又蕴含着提炼精华的意味，让我一下就记住了。


## 4、人在闭环

In [63]:
from langchain_community.tools.tavily_search import TavilySearchResults
web_search = TavilySearchResults(max_results=2)
tools = [web_search]

In [64]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# 创建 Agent，接入 HumanInTheLoopMiddleware
agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # 拦截 Tavily 搜索工具执行前，要求人工确认
                "tavily_search_results_json": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                    "description": lambda tool_name, tool_input, state: (
                        f"🔍 模型准备执行 Tavily 搜索：'{tool_input.get('query', '')}'"
                    ),
                }
            },
            description_prefix="⚠️ 工具执行需要人工审批"
        )
    ],
)

In [65]:
result = agent.invoke(
    {"messages": "LangChain1.0的公共有哪些？"},
    config
)

In [66]:
result

{'messages': [HumanMessage(content='LangChain1.0的公共有哪些？', additional_kwargs={}, response_metadata={}, id='9aa9f3bd-2ba8-4050-9a8a-63832800be06'),
  AIMessage(content='目前我无法直接查询到LangChain 1.0的公共仓库列表信息。要获取LangChain 1.0的公共仓库，您可以：\n\n1. 直接访问LangChain的官方GitHub页面：https://github.com/langchain-ai\n2. 在GitHub上搜索"langchain"相关仓库\n3. 查看LangChain官方文档，通常会列出相关的公共仓库和组件\n\nLangChain作为一个开源框架，通常会有多个相关的仓库，比如核心库、社区贡献、示例代码等。建议您通过上述方式获取最新的仓库列表信息。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 192, 'total_tokens': 312, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 64}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'c0c6741b-45b2-476b-84b9-66a9fa672eb9', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--af5edb37-834f-469a-a675-d4a4ab374f3d-0', us